In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

os.chdir('..')

# WritingProject

## 全自动生成：使用默认的智谱AI

In [2]:
from langchain_chinese import TreeContent, WritingTask

# 使用默认的智谱AI推理
# 你需要准备好智谱AI的APIKEY
wp = WritingTask(task_mode="auto")
wp.run("我今天扒了二牛的裤子，然后他一整天都不理我，帮我写一封1000字道歉信，我希望和他继续做好朋友")

Task Mode: auto
Focus from: root@input
-------------------- Todos --------------------
<位置：root>
```json
{
    "类型": "root",
    "总字数要求": 1000,
    "标题名称": "致二牛的诚挚道歉信",
    "扩写指南": "本信旨在对扒裤子事件进行深刻反省和诚恳道歉，需表达出对二牛情绪的理解、对友谊的珍视以及对未来友好相处的期望。在信中可提及事件发生的具体情况，自身当时的错误心态，以及现在的悔过态度。此外，可以适当描述一下你和二牛之间的美好回忆，强调不想因一时冲动而失去这份友谊。"
}
```
-------------------- Todos --------------------
<位置：root> 1000字以内 | 《致二牛的诚挚道歉信》
👤[auto]:  请帮我扩写《致二牛的诚挚道歉信》, 字数不超过1000字，扩写依据为：本信旨在对扒裤子事件进行深刻反省和诚恳道歉，需表达出对二牛情绪的理解、对友谊的珍视以及对未来友好相处的期望。在信中可提及事件发生的具体情况，自身当时的错误心态，以及现在的悔过态度。此外，可以适当描述一下你和二牛之间的美好回忆，强调不想因一时冲动而失去这份友谊。
```json
{
    "类型": "outline",
    "标题名称": "致二牛的诚挚道歉信",
    "大纲列表": [
        {
            "总字数要求": 250,
            "标题名称": "事件的回顾与反省",
            "扩写指南": "详细描述扒裤子事件的发生经过，自身为何会做出这样的行为，以及现在的反省和认识到的错误。"
        },
        {
            "总字数要求": 250,
            "标题名称": "理解二牛的情绪",
            "扩写指南": "表达对二牛当时和事后情绪的理解，认识到自己的行为对他造成了多大的困扰和伤害。"
        },
        {
            "总字数要求": 250,
            "标题名称": "珍视友谊的表白",
   

## 每一步都询问：使用默认的智谱AI

In [2]:
from langchain_chinese import TreeContent, WritingTask

# 使用默认的智谱AI推理
wp = WritingTask(task_mode="askme")
wp.run()

Task Mode: askme
Focus from: root@input
-------------------- root@input --------------------
-------------------- Todos --------------------
<root> None字以内 | 《None》



👤:  帮我写一个小故事


-------------------- AI for root --------------------
```json
{
    "类型": "root",
    "总字数要求": 1000,
    "标题名称": "山涧奇遇",
    "扩写指南": "故事背景设置在一片神秘的山林中，主人公在山中迷路，意外发现一处山涧，并在那里遇到了奇特的生物。主人公需具备勇敢和智慧的品质。建议在故事中包含对山涧美景的描绘，以及与生物之间的互动和感悟。"
}
```



👤:  ok


-------------------- root@output --------------------
-------------------- Todos --------------------
<root> 1000字以内 | 《山涧奇遇》
-------------------- Move To: root@output --------------------
👤[auto]:  请帮我扩写《山涧奇遇》, 字数不超过1000字，扩写依据为：故事背景设置在一片神秘的山林中，主人公在山中迷路，意外发现一处山涧，并在那里遇到了奇特的生物。主人公需具备勇敢和智慧的品质。建议在故事中包含对山涧美景的描绘，以及与生物之间的互动和感悟。
-------------------- AI for root --------------------
```json
{
    "类型": "outline",
    "标题名称": "山涧奇遇",
    "大纲列表": [
        {
            "总字数要求": 200,
            "标题名称": "迷路的旅人",
            "扩写指南": "描述主人公在神秘山林中迷路的情景，展现主人公的困惑和勇敢面对困境的态度。"
        },
        {
            "总字数要求": 300,
            "标题名称": "发现山涧",
            "扩写指南": "描绘主人公意外发现的山涧美景，包括清澈的溪流、瀑布、奇石等自然景观。"
        },
        {
            "总字数要求": 200,
            "标题名称": "奇特的生物",
            "扩写指南": "描述主人公在山涧遇到的奇特生物，以及它们的特征和主人公的初次反应。"
        },
        {
            "总字数要求": 300,
            "标题名称": "智慧与勇气",
            "扩写指南": "展现主人公如何运用智慧和勇气与生物互动，并从中学到的重要教训或感悟。"
        }
    ]
}
```
---------------

...

-------------------- AI for root --------------------
```json
{
    "类型": "outline",
    "标题名称": "山涧奇遇",
    "大纲列表": [
        {
            "总字数要求": 500,
            "标题名称": "迷路与发现",
            "扩写指南": "描述主人公在神秘山林中迷路的过程，以及他如何意外发现山涧的经过。包括对山涧美景的初步描绘和主人公的内心感受。"
        },
        {
            "总字数要求": 500,
            "标题名称": "与生物的互动",
            "扩写指南": "详述主人公在山涧遇到的奇特生物，他们的互动过程，以及主人公如何凭借勇敢和智慧与生物相处，并从中得到的感悟。"
        }
    ]
}
```
-------------------- root@output --------------------
-------------------- Todos --------------------
<root> 1000字以内 | 《山涧奇遇》



👤:  ok


-------------------- Outlines Done for root --------------------
-------------------- 20240508.180514.00001.408@output --------------------
-------------------- Todos --------------------
<20240508.180514.00001.408> 500字以内 | 《迷路与发现》
<20240508.180514.00002.626> 500字以内 | 《与生物的互动》
-------------------- Move To: 20240508.180514.00001.408@output --------------------
👤[auto]:  请帮我扩写《迷路与发现》, 字数不超过500字，扩写依据为：描述主人公在神秘山林中迷路的过程，以及他如何意外发现山涧的经过。包括对山涧美景的初步描绘和主人公的内心感受。
-------------------- AI for 20240508.180514.00001.408 --------------------
```json
{
    "类型": "paragraph",
    "标题名称": "迷路与发现",
    "详细内容": "主人公独自踏入那片神秘的山林，阳光透过繁茂的树叶，斑驳地洒在蜿蜒的山径上。起初，他还信心满满，但随着时间的推移，他发现自己迷路了。在反复尝试寻找出路无果后，他有些焦虑和恐慌。然而，正是在这种困境中，他意外地发现了一处隐蔽的山涧。水流潺潺，清澈见底，周围是翠绿的植被和五彩斑斓的花朵。主人公被这美景深深吸引，心中的焦虑渐渐消散。他站在山涧边，感受着山风拂过脸颊，心中涌起一股对大自然的敬畏和赞叹。",
    "内容摘要": "主人公在神秘的山林中迷路，意外发现了一处美丽的山涧，感受到了大自然的魅力。这片山涧成了他心灵上的慰藉，也预示着接下来的奇遇。"
}
```
-------------------- 20240508.180514.00001.408@output --------------------
-------------------- Todos ---------------


👤:  quit


-------------------- quit --------------------


In [3]:
wp.run()

-------------------- 20240508.180514.00001.408@output --------------------
-------------------- Todos --------------------
<20240508.180514.00001.408> 500字以内 | 《迷路与发现》
<20240508.180514.00002.626> 500字以内 | 《与生物的互动》
👤[auto]:  请帮我扩写《迷路与发现》, 字数不超过500字，扩写依据为：描述主人公在神秘山林中迷路的过程，以及他如何意外发现山涧的经过。包括对山涧美景的初步描绘和主人公的内心感受。
-------------------- AI for 20240508.180514.00001.408 --------------------
```json
{
    "类型": "paragraph",
    "标题名称": "迷路与发现",
    "详细内容": "主人公在神秘的山林中探索，原本期待的冒险之旅却因迷路而变得紧张起来。他试图按照地图上的标记前进，但密集的树木和不断变化的自然地形让他的努力变得徒劳。随着天色渐暗，他开始焦虑，步伐也变得紊乱。就在这时，他意外地闯入了一片开阔地，眼前出现了一道美丽的山涧。水流如练，从山石间倾泻而下，水花四溅，周围的花草随风轻摆，仿佛在欢迎他的到来。主人公心中的恐惧瞬间被这美景所取代，他感到一种前所未有的平静和喜悦，内心深处似乎有某种东西被唤醒了。",
    "内容摘要": "主人公在神秘山林中迷路，偶然发现了一处隐蔽的山涧。山涧的美景让他忘记了迷路的恐惧，他的内心经历了从焦虑到平静的转变，这预示着他与这片山林更深层次的联系即将展开。"
}
```



👤:  再多一些文字


-------------------- AI for 20240508.180514.00001.408 --------------------
```json
{
    "类型": "paragraph",
    "标题名称": "迷路与发现",
    "详细内容": "主人公踏入了被称为「遗忘之地」的神秘山林，这里古木参天，藤蔓缠绕，每一步都充满了未知。他原本自信满满，但随着时间的流逝，他发现自己迷失在了一个由绿色迷宫构成的世界中。在不断的寻找与尝试中，他无意间闯入了一片隐蔽的山涧。这里的景色美得令人窒息：瀑布如银河倾泻，水潭如碧玉镶嵌，周围的花草散发着淡淡的香气。主人公的内心由恐慌转为震撼，他感受到了一种与世隔绝的宁静和自然的伟力。这种宁静与力量，让他的心灵得到了前所未有的洗涤，也激发了他勇敢和智慧的潜能。",
    "内容摘要": "主人公在神秘山林中意外发现了一处美丽的山涧，这一发现让他从迷路的恐慌中解脱出来，感受到了大自然的宁静与力量。这段经历为他接下来的奇遇奠定了基础，展现了他勇敢和智慧的品质。"
}
```

请注意，这是《山涧奇遇》中第一部分“迷路与发现”的扩写。第二部分“与生物的互动”需要根据您的指示继续扩写。如果您需要，我可以继续提供相应的内容。
-------------------- 20240508.180514.00001.408@output --------------------
-------------------- Todos --------------------
<20240508.180514.00001.408> 500字以内 | 《迷路与发现》
<20240508.180514.00002.626> 500字以内 | 《与生物的互动》



👤:  不要用主人公字眼，用具体人名


-------------------- AI for 20240508.180514.00001.408 --------------------
```json
{
    "类型": "paragraph",
    "标题名称": "迷路与发现",
    "详细内容": "李明，一个充满好奇心和冒险精神的青年，踏入了被称为「遗忘之地」的神秘山林。他本想一探究竟，却未曾想到自己会在这片绿意盎然中迷失方向。他在森林中穿梭，试图寻找出路，但四周的景象似乎在不断变幻，让他愈发困惑。就在李明开始感到焦虑时，他意外地走到了一处山涧边。眼前的景色让他惊叹：清澈的溪流在阳光下闪烁，瀑布如丝般顺滑，周围的花草散发着迷人的香气。李明的心中涌起一股宁静，他感到自己的勇敢和智慧在这片自然奇观中得到启发。",
    "内容摘要": "李明在神秘的山林中迷路，偶然发现了一处隐蔽的山涧。这里的景色让他忘记了迷路的恐惧，他的内心经历了从焦虑到平静的转变，勇敢和智慧的品质在这片自然奇观中得到激发。"
}
```
-------------------- 20240508.180514.00001.408@output --------------------
-------------------- Todos --------------------
<20240508.180514.00001.408> 500字以内 | 《迷路与发现》
<20240508.180514.00002.626> 500字以内 | 《与生物的互动》


...

-------------------- 20240508.180514.00002.626@output --------------------
-------------------- Todos --------------------
<20240508.180514.00002.626> 500字以内 | 《与生物的互动》
-------------------- Move To: 20240508.180514.00002.626@output --------------------
👤[auto]:  请帮我扩写《与生物的互动》, 字数不超过500字，扩写依据为：详述主人公在山涧遇到的奇特生物，他们的互动过程，以及主人公如何凭借勇敢和智慧与生物相处，并从中得到的感悟。
-------------------- AI for 20240508.180514.00002.626 --------------------
```json
{
    "类型": "paragraph",
    "标题名称": "与生物的互动",
    "详细内容": "李明在山涧边遇见了一种他从未见过的生物，它们有着透明的翅膀，身体呈彩虹色，被水珠沾湿后闪烁着光芒。这些生物似乎并不惧怕人类，反而对李明的到来充满好奇。起初，李明有些害怕，担心这些奇特生物可能会有危险。但他很快冷静下来，细心观察它们的行为，试图理解它们。通过模仿它们的动作，他逐渐与这些生物建立起一种特殊的沟通方式。他发现，当自己展现出平静和善意时，这些生物也会给予他同样的回应。在互动的过程中，李明意识到，勇敢不仅仅是面对恐惧，更是在未知面前保持冷静和理智。而智慧则是通过观察和理解，去适应和融入周围的环境。",
    "内容摘要": "在山涧的奇遇中，李明遇到了一群奇特的彩虹色生物，通过勇敢和智慧的互动，他学会了与自然和谐相处的重要性，理解了勇敢和智慧在适应环境中的意义。"
}
```
-------------------- 20240508.180514.00002.626@output --------------------
-------------------- Todos --------------------
<20240508.180514.00002.626> 500字以内 | 《与


👤:  字多些，1000字吧


-------------------- AI for 20240508.180514.00002.626 --------------------
```json
{
    "类型": "paragraph",
    "标题名称": "山涧奇遇",
    "详细内容": "李明在神秘的山林中迷路了。起初，他焦虑不安，四周是茂密的树林和蜿蜒的小径，让他辨不清方向。但就在他几乎要放弃的时候，一阵清脆的流水声吸引了他的注意。跟随声音，他意外发现了一处隐蔽的山涧。这里的景色美得让他忘记了迷路的恐惧：瀑布从高处倾泻而下，水花四溅，阳光透过树叶的缝隙，在水面上洒下斑驳的光影。他感到内心从焦虑到平静的转变，仿佛这片自然奇观激发了他内在的勇敢和智慧。

在与山涧美景相伴的过程中，李明注意到了一群奇特的生物。它们有着透明的翅膀，身体呈现出斑斓的色彩，就像是从梦境中走出来的精灵。起初，这些生物对他保持警惕，但李明没有退缩，他试图用各种方式与它们沟通。他模仿它们的动作，轻声细语地与它们交谈，甚至尝试通过肢体语言表达自己的善意。随着时间的推移，生物们开始放松警惕，与李明展开互动。

在与这些生物的互动中，李明学会了尊重和耐心。他发现，这些生物对于环境的变化极为敏感，它们能通过微妙的信号传递信息。李明开始反思人类社会中的沟通方式，意识到在自然面前，人类还有许多需要学习的地方。他的勇敢不仅体现在面对未知时的勇气，更在于他愿意放下人类的优越感，去理解和适应这些生物的习性。

这次奇遇让李明深刻地领悟到，勇敢和智慧并不仅仅是人类的专利，它们是所有生物在生存斗争中逐渐形成的品质。他开始思考如何将这种与自然和谐相处的方式应用到日常生活中，如何在人类社会中也展现出这样的尊重和理解。",
    "内容摘要": "在神秘的山林中，李明迷路后偶然发现了一处美丽的山涧，并在那里遇到了一群奇特的生物。通过与它们的互动，李明学会了勇敢和智慧地与自然相处，体会到了尊重和耐心的真谛，他希望将这些感悟应用到人类社会中去。"
}
```
-------------------- 20240508.180514.00002.626@output --------------------
-------------------- Todos --------------------
<20240508.180514.


👤:  ok


-------------------- None --------------------
---------------------------------------- 
All Complete!
-------------------- Move To: None --------------------


## 查看记忆和对话历史

In [26]:
[k for k in wp.memory._shorterm_memory_store]

['root',
 '20240508.180514.00001.408',
 '20240508.180514.00002.626',
 (('session_id', 'root'),),
 (('session_id', '20240508.180514.00002.626'),)]

In [27]:
for x in wp.memory.get_shorterm_memory(session_id="20240508.180514.00002.626").chat_memory.messages:
    print("-"*40)
    print(x.content)

----------------------------------------
请帮我扩写《与生物的互动》, 字数不超过500字，扩写依据为：详述主人公在山涧遇到的奇特生物，他们的互动过程，以及主人公如何凭借勇敢和智慧与生物相处，并从中得到的感悟。
----------------------------------------
```json
{
    "类型": "paragraph",
    "标题名称": "与生物的互动",
    "详细内容": "李明在山涧边遇见了一种他从未见过的生物，它们有着透明的翅膀，身体呈彩虹色，被水珠沾湿后闪烁着光芒。这些生物似乎并不惧怕人类，反而对李明的到来充满好奇。起初，李明有些害怕，担心这些奇特生物可能会有危险。但他很快冷静下来，细心观察它们的行为，试图理解它们。通过模仿它们的动作，他逐渐与这些生物建立起一种特殊的沟通方式。他发现，当自己展现出平静和善意时，这些生物也会给予他同样的回应。在互动的过程中，李明意识到，勇敢不仅仅是面对恐惧，更是在未知面前保持冷静和理智。而智慧则是通过观察和理解，去适应和融入周围的环境。",
    "内容摘要": "在山涧的奇遇中，李明遇到了一群奇特的彩虹色生物，通过勇敢和智慧的互动，他学会了与自然和谐相处的重要性，理解了勇敢和智慧在适应环境中的意义。"
}
```
----------------------------------------
字多些，1000字吧
----------------------------------------
```json
{
    "类型": "paragraph",
    "标题名称": "山涧奇遇",
    "详细内容": "李明在神秘的山林中迷路了。起初，他焦虑不安，四周是茂密的树林和蜿蜒的小径，让他辨不清方向。但就在他几乎要放弃的时候，一阵清脆的流水声吸引了他的注意。跟随声音，他意外发现了一处隐蔽的山涧。这里的景色美得让他忘记了迷路的恐惧：瀑布从高处倾泻而下，水花四溅，阳光透过树叶的缝隙，在水面上洒下斑驳的光影。他感到内心从焦虑到平静的转变，仿佛这片自然奇观激发了他内在的勇敢和智慧。

在与山涧美景相伴的过程中，李明注意到了一群奇特的生物。它们有着透明的翅膀，身体呈现出斑斓的色彩，就像是从梦境中走出来的精灵。起初，这些生物对他保持

In [28]:
wp.print_text()

《山涧奇遇》
总字数要求约1000字；已完成。
扩写指南 >>> 故事背景设置在一片神秘的山林中，主人公在山中迷路，意外发现一处山涧，并在那里遇到了奇特的生物。主人公需具备勇敢和智慧的品质。建议在故事中包含对山涧美景的描绘，以及与生物之间的互动和感悟。

1 迷路与发现
 李明，一个充满好奇心和冒险精神的青年，踏入了被称为「遗忘之地」的神秘山林。他本想一探究竟，却未曾想到自己会在这片绿意盎然中迷失方向。他在森林中穿梭，试图寻找出路，但四周的景象似乎在不断变幻，让他愈发困惑。就在李明开始感到焦虑时，他意外地走到了一处山涧边。眼前的景色让他惊叹：清澈的溪流在阳光下闪烁，瀑布如丝般顺滑，周围的花草散发着迷人的香气。李明的心中涌起一股宁静，他感到自己的勇敢和智慧在这片自然奇观中得到启发。
2 与生物的互动
 李明在神秘的山林中迷路了。起初，他焦虑不安，四周是茂密的树林和蜿蜒的小径，让他辨不清方向。但就在他几乎要放弃的时候，一阵清脆的流水声吸引了他的注意。跟随声音，他意外发现了一处隐蔽的山涧。这里的景色美得让他忘记了迷路的恐惧：瀑布从高处倾泻而下，水花四溅，阳光透过树叶的缝隙，在水面上洒下斑驳的光影。他感到内心从焦虑到平静的转变，仿佛这片自然奇观激发了他内在的勇敢和智慧。

在与山涧美景相伴的过程中，李明注意到了一群奇特的生物。它们有着透明的翅膀，身体呈现出斑斓的色彩，就像是从梦境中走出来的精灵。起初，这些生物对他保持警惕，但李明没有退缩，他试图用各种方式与它们沟通。他模仿它们的动作，轻声细语地与它们交谈，甚至尝试通过肢体语言表达自己的善意。随着时间的推移，生物们开始放松警惕，与李明展开互动。

在与这些生物的互动中，李明学会了尊重和耐心。他发现，这些生物对于环境的变化极为敏感，它们能通过微妙的信号传递信息。李明开始反思人类社会中的沟通方式，意识到在自然面前，人类还有许多需要学习的地方。他的勇敢不仅体现在面对未知时的勇气，更在于他愿意放下人类的优越感，去理解和适应这些生物的习性。

这次奇遇让李明深刻地领悟到，勇敢和智慧并不仅仅是人类的专利，它们是所有生物在生存斗争中逐渐形成的品质。他开始思考如何将这种与自然和谐相处的方式应用到日常生活中，如何在人类社会中也展现出这样的尊重和理解。


## 全自动生成：更换为GPT4

In [29]:
from langchain_openai import ChatOpenAI

# 使用 OpenAI/ChatGPT 推理
wp = WritingTask(task_mode="auto")
wp.run(llm = ChatOpenAI(model_name="gpt-4-turbo"))

Task Mode: auto
Focus from: root@input
-------------------- root@input --------------------
-------------------- Todos --------------------
<root> None字以内 | 《None》



👤:  我老婆今天跟我生气了，一起生活了20多年，我想哄哄她，帮我写一封道歉信


-------------------- AI for root --------------------
```json
{
    "类型": "root",
    "总字数要求": 400,
    "标题名称": "道歉信",
    "扩写指南": "写作风格应该诚恳、真诚，表达出内心的歉意和对她的重视。可以回顾过去的美好时光，承认自己的错误，并表达出改正的决心。建议在信中适当表达对她的爱和感激，以及对未来更好生活的期许。"
}
```
-------------------- root@output --------------------
-------------------- Todos --------------------
<root> 400字以内 | 《道歉信》
-------------------- Move To: root@output --------------------
👤[auto]:  请帮我扩写《道歉信》, 字数不超过400字，扩写依据为：写作风格应该诚恳、真诚，表达出内心的歉意和对她的重视。可以回顾过去的美好时光，承认自己的错误，并表达出改正的决心。建议在信中适当表达对她的爱和感激，以及对未来更好生活的期许。
-------------------- AI for root --------------------
```json
{
    "类型": "paragraph",
    "标题名称": "道歉信",
    "详细内容": "亲爱的（妻子的姓名），我写这封信是因为我深深地意识到我伤害了你，我感到十分抱歉。我知道我所犯下的错误给你带来了痛苦和失望，而这一切都是我不应该让你承受的。回顾过去的日子，我们一起经历了太多美好的时光，那些幸福的回忆永远珍藏在我心中。我不想因为我的过失而破坏我们的感情，更不愿看到你因此受到伤害。

我承认，我在某些时候没有站在你的角度去考虑问题，我对你的体贴和理解不够，这是我的疏忽。但我保证，我会从这次错误中吸取教训，真诚地改正自己的不足。我会更加用心地倾听你的心声，更加关注你的感受，让我们的关系更加美好和稳固。

我深深地爱着你，你是我的全部。我感激你一直以来的支持和包容，没有你，我的生活将是一片空白。我希望我们能够一起面对生活中的挑战，共同创造更美好的未来。我发誓会为了我

In [11]:
wp.print_lines()

1 风雨之夜的到来
在一个暴风雨的夜晚，约翰和汤姆决心探索镇上那栋有名的老房子。这座房子因其神秘的过去和诡异的传说而闻名。据说，几十年前，一位富有的商人在此神秘失踪，自那以后，无数的探险者和好奇者试图揭开房子的秘密，却无一例外地被各种怪异的事件和不幸的命运所阻挠。约翰和汤姆对超自然现象充满了兴趣，他们相信，通过他们的智慧和勇气，可以揭示这座老房子隐藏的真相。

他们的征途并不顺利。天色逐渐暗沉，狂风大作，雷声隆隆，仿佛大自然也在警告他们不要前进。当他们接近房子时，几个当地人劝他们回头，告诉他们许多人因好奇心而遭受不幸。但约翰和汤姆的决心未被动摇，他们相信这是一次生命中难得的冒险，即使是最恶劣的天气和最棘手的警告也阻挡不了他们的脚步。

2 秘密的低语声


2.1 进入老房子
在风雨交加的夜晚，约翰和汤姆站在那座传说中的老房子前，怀着忐忑不安的心情，他们推开了那扇沉重的前门。门吱嘎作响，仿佛在抗议着这对未被邀请的客人。一进入，一股阴冷及潮湿的空气迎面扑来，使人不寒而栗。房内光线昏暗，只有外面闪电的光芒偶尔透进来，照亮了房间里陈旧的家具和满墙的蛛网。两人试图适应这光线，四处观望，内心的恐惧与好奇心在此刻交织在一起。

2.2 低语声的发现
当约翰和汤姆勇敢地深入老房子的阴暗走廊时，一阵低沉而不连续的低语声突然在空气中回荡。这声音似乎从无处而来，却又无处不在，使得两人不由自主地停下脚步，屏息凝听。他们的眼神交换了一个忐忑的眼色，相互点头后，悄无声息地沿着声音的大致方向前进。汤姆轻轻地把耳朵贴近发霉的墙壁，试图捕捉那若有若无的低语，而约翰则试图通过手电筒的微弱光线，寻找可能的声源。在这种未知与恐惧的驱使下，他们小心翼翼地一步步接近那神秘的低语声。

2.3 追踪与恐惧
随着每一步深入，房子的古老木地板在约翰和汤姆的脚下吱吱作响。隐约的低语声引导他们穿过长长的走廊，每当他们靠近，一阵风似乎从窗外掠过，使得厚重的窗帘无声无息地摇晃。这些动静让房间笼罩在一种不安的气氛中，他们的心跳加速，恐惧与好奇心在胸中交织，推动他们继续前行，尽管内心深处不断有声音告诫他们退后。

3 揭开真相的时刻
最终，约翰和汤姆在房子的最深处，发现了低语声的来源。他们不禁停下脚步，目瞪口呆地望着眼前的场景。在昏暗的灯光下，一张古老的书桌上摆放着一个古怪的箱子，发出微弱的低语声。随着他们的靠近，低语声渐渐

In [10]:
print(wp.root_content.get_outlines())

1 第一章：重名的阴影 
介绍主要人物和背景，发现小说中的名字与现实中的同学相同，开始发生第一个意外事件。
1.1 开篇介绍 
故事背景：静枫中学，主要人物：李晓风，赵瑞，林雨晨，王子豪。
1.2 日常变奏 
主要人物李晓风在图书馆发现一本小说《死亡名单》，里面的角色名字与现实中的同学一模一样，引发了一连串的不安与怀疑。
1.3 阴影初现 
第一个意外事件：王子豪在打扫卫生时，无缘无故从楼梯上摔下，头部受创。这一事件引发主要人物们的不安与怀疑。
1.4 人心惶惶 
意外事件后，主要人物和周围同学都感受到了巨大的心理压力和恐惧，整个学校的氛围变得紧张不安。
2 第二章：恐惧蔓延 
角色们开始感到不安，第二个和第三个重名的同学也遭遇不幸，大家陷入恐慌。
2.1 2.1 不祥的征兆 
第二个重名同学赵瑞频繁遭遇诡异事件，李晓风和林雨晨对此感到担忧和恐惧，担心他会成为下一个不幸者。
2.2 2.2 恐慌的蔓延 
第二个重名同学赵瑞遭遇不幸事件，引发学校内恐慌情绪蔓延，同学们纷纷避开与小说角色重名的人，学校氛围变得沉重。主要人物们意识到必须找到解决方法。
2.3 2.3 灾难的连续 
第三个重名同学林雨晨在类似情况下遭遇不幸，导致主要角色们的恐慌达到顶点。人物：林雨晨，李晓风，王子豪。背景：静枫中学。设定：小说《死亡名单》中的角色与现实中的同学重名，引发连续不幸事件。
2.4 2.4 陷入混乱 
静枫中学的恐慌氛围，主要人物调查诅咒但陷入混乱，其他同学避开他们，调查变得更加困难。
3 第三章：揭秘行动 
主要角色决定调查这一系列事件的背后原因，发现小说的存在与一个古老的诅咒有关。
3.1 3.1 调查启动 
主要人物李晓风、赵瑞、林雨晨在图书馆展开首次调查会议，制定调查计划，分工合作，开始揭秘行动。涉及实体：李晓风、赵瑞、林雨晨、静枫中学图书馆。设定：揭开《死亡名单》背后的谜团，分工合作进行调查。
3.2 3.2 线索搜集 
人物：李晓风、赵瑞、林雨晨。地点：静枫中学图书馆。情节：角色们搜集到小说《死亡名单》与古老诅咒的联系。背景设定：静枫中学图书馆藏有关于诅咒的线索，学校可能隐藏着揭开诅咒的秘密。
3.3 3.3 诅咒揭秘 
在本段中，主要角色们在图书馆的古老资料中发现了《死亡名单》小说与一个古老诅咒的联系。这个诅咒源自几百年前一位被背叛的术士，诅咒的名字会传递并给继承者带